In [1]:
from src.collision import CollisionDetector
obj_folder = "/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation"

In [3]:
t = CollisionDetector(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv")

PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation/240905-1616_seed_session28.toml') generated an exception: File /Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation/240905-1616_seed_session28.toml ignored, not .dae or .obj file
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation/.DS_Store') generated an exception: File /Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation/.DS_Store ignored, not .dae or .obj file


In [6]:
t = CollisionDetector(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv", obj_ref_frame=2599, obj_transform_toml=f"{obj_folder}/240905-1616_seed_session28.toml")

In [7]:
from src.visual_widget import MultiViewer
V = MultiViewer([t.animal], [t.obj], 0)

ERROR:src.animal:No rays present for animal at frame 843
ERROR:src.animal:No rays present for animal at frame 845
ERROR:src.animal:No rays present for animal at frame 846
ERROR:src.animal:No rays present for animal at frame 847
ERROR:src.animal:No rays present for animal at frame 854
ERROR:src.animal:No rays present for animal at frame 855
ERROR:src.animal:No rays present for animal at frame 857
ERROR:src.animal:No rays present for animal at frame 843
ERROR:src.animal:No rays present for animal at frame 845
ERROR:src.animal:No rays present for animal at frame 846
ERROR:src.animal:No rays present for animal at frame 847
ERROR:src.animal:No rays present for animal at frame 854
ERROR:src.animal:No rays present for animal at frame 855
ERROR:src.animal:No rays present for animal at frame 857


AttributeError: 'NoneType' object has no attribute 'object_space'

In [ ]:
V.scene_widget.scene.graph.show()